In [1]:
import requests
import random
import re

In [2]:
#@title board utils
from os import kill
def generate_board(n = 25, lang = "eng", c = 5, k = 1):
  '''
  inputs:
    n (int): number of words
    lang (str): language of dictionary (must be on the github folder)
    c (int): number of colored words for each team
    k (int): number of killer words

  returns a dictionary such that dict[word] = color
  '''

  url = f"https://raw.githubusercontent.com/mich1803/Codenames-LLM/main/wordlists/{lang}.txt"

  try:
    # Fetch the content of the file
    response = requests.get(url)
  except:
    raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")
  words = response.text.splitlines()

  # Select n unique random words
  random_words = random.sample(words, n)

  # Select indices for the colors
  indices = list(range(n))
  blue_indices = random.sample(indices, c)
  remaining_indices = [i for i in indices if i not in blue_indices]
  red_indices = random.sample(remaining_indices, c)
  remaining_indices = [i for i in remaining_indices if i not in red_indices]
  black_index = random.sample(remaining_indices, k)

  colors = [
      "blue" if i in blue_indices else
      "red" if i in red_indices else
      "killer" if i in black_index else
      "neutral"
      for i in range(25)
      ]

  # Create a dictionary with list comprehension
  word_color_dict = {random_words[i]: colors[i] for i in range(25)}
  return word_color_dict

def board4prompt(word_color_dict, lang = "eng", master = False):
  '''
  inputs:
    word_color_dict(dict): board dict generated from generate_board()
    lang (str): language of prompt

  return a string version for the prompt for LLMs
  '''

  if lang == "eng":
    init = "The actual word board is: \n"
  elif lang == "ita":
    init = "La board al momento è: \n"
  else: raise Exception("ERROR: The language entered is wrong or has no dictionary in the github folder")

  if master:
    return init + (" | ". join([f"{i} ({word_color_dict[i]})" for i in word_color_dict])) + "\n"
  else:
    return init + (" | ".join(word_color_dict)) + "\n"

In [3]:
board = generate_board(lang = "ita")
board_prompt = board4prompt(board, lang="ita", master = True)
print(board_prompt)

La board al momento è: 
MERITO (killer) | SIGNORE (neutral) | SCUOLA (blue) | PUNTA (red) | COMPAGNIA (neutral) | COLPO (neutral) | SOCIETÀ (neutral) | BISOGNO (red) | PAROLA (neutral) | PROCESSO (red) | PESO (blue) | CARATTERE (red) | ALBERGO (neutral) | CAVALLO (blue) | SUCCESSO (blue) | VALORE (red) | POLITICA (neutral) | PUNTO (neutral) | SPIRITO (neutral) | CORSO (neutral) | ETÀ (blue) | PAESE (neutral) | RISPETTO (neutral) | INDUSTRIA (neutral) | TIPO (neutral)



cose da implementare ancora per la gestione del gioco:
*   funzione per leggere il prompt di indizio del master LLM (magari chiedere all'LLM di scrivere la risposta in un modo particolare nel prompt, esempio: dai l'indizio di una parola e un numero nel formato {[(parola)(numero)]})
*   funzione per leggere il prompt di risposta dell'indovino LLM
*   funzione per la partita: finisce se una squadra prende tutte le sue carte o se la carta killer viene scoperta, la partita restituisce: una tupla (vincitore, metodo di vittoria, round)
*   fare le statistiche su quale metodo di vittoria è più frequente, in quanti round finisce mediamente la partita, confrontarli tra LLM, lingue, numero di carte ecc.



In [38]:
#@title prompts

START_master = {"ita": "Stiamo giocando al gioco da tavola NOMI IN CODICE, il tuo ruolo sarà quello del master. \n " +
                        "Data una lista con delle parole, accompagnate da un colore, devi cercare di far indovinare agli indovini della tua squadra, solo le parole con il colore della tua squadra. \n" +
                        "Devi fornirgli un indizio di una parola (che non sia nella board) e un numero che rappresenta il  numero di parole relazionate a quell'indizio. \n" +
                        "ATTENTO: se la tua squadra indovina la parola 'killer' perde istantaneamente. \n" +
                        "Dai la risposta nel formato: {(inidizio di una parola),(numero)}. Sei pronto? \n",

                "eng": "We are playing the board game CODENAMES, and your role will be that of the spymaster. \n" +
                       "Given a list of words, each accompanied by a color, you must try to make the guessers on your team guess only the words with the color of your team. \n" +
                       "You must give them a one-word clue (which is not on the board) and a number that represents the number of words related to that clue. \n" +
                       "BEWARE: if your team says the 'killer' word loses instantly. \n" +
                       "Provide your answer in the format: {(one-word clue),(number)}. Are you ready? \n"}

team4prompt = {('eng','red'): "You are on the RED team. \n",
               ('eng','blue'): "You are on the BLUE team. \n",
               ('ita', 'red'): "Fai parte della squadra ROSSA. \n",
               ('ita', 'blue'): "Fai parte della squadra BLU. \n"}

FORMAT_master = {"ita": "Scrivi la tua risposta nel formato: {(inidizio di una parola),(numero)}",
                  "eng": "Write your answer in the format: {(one-word clue),(number)}"}

#IL PROMPT INIZIALE DEL MASTER SARA': START_master_1 + master_board + START_master_2 + team4prompt + FORMAT_master

START_guesser = {"ita": "Stiamo giocando al gioco da tavola NOMI IN CODICE, il tuo ruolo sarà quello del guesser. \n" +
                        "Date le parole sulla board e all'indizio del tuo master devi cercare di indovinare le parole della tua squadra. \n "+
                        "Se indovini e il tuo master ha dato un numero maggiore di uno puoi fare un altro tentativo.\n Scrivi la tua risposta nel formato: {risposta}. \n Sei pronto? \n",
                 "eng": "We are playing the board game CODENAMES, and your role will be that of the guesser. \n"+
                        "Given the words on the board and the clue from your spymaster, you must try to guess your team's words. \n "+
                        "If you guess correctly and your spymaster has given a number greater than one, you can make another attempt.\n Write your answer in the format: {one-word guess}. \n Are you ready? \n"}

FORMAT_guesser = {"ita": "Scrivi la tua risposta nel formato: {risposta}",
                  "eng": "Write your answer in the format: {guess}"}

MtoG_clue = {
    "ita": "Il tuo master ha dato l'indizio: ",
    "eng": "The master gave the clue: "
}

TURN_guesser = {
    "ita": "è il tuo turno di indovinare. \n",
    "eng": "It's yor turn to guess. \n"
}

CORRECT_guesser = {
    "ita": "Hai indovinato una parola per la tua squadra! \n Ora fai un altro tentativo, ti ricordo che: \n",
    "eng": "You guessed it right! \n Now try again, I remind you that: \n"
}

TURN_master = {
    "ita": "è il tuo turno, ti ricodo che: Devi fornirgli un indizio di una parola (che non sia nella board) e un numero che rappresenta il  numero di parole relazionate a quell'indizio. \n",
    "eng": "it's your turn, i remind you that: You must give them a one-word clue (which is not on the board) and a number that represents the number of words related to that clue. \n"
}

reminder = {
    "ita": "Ti ricordo che: \n",
    "eng": "I remind you that: \n"
}




In [44]:
#@title text-handling utils
def read_clue(answer):
  # Regular expression to match the format "{(one-word clue),(number)}"
    match = re.search(r"\{\((\w+)\),\((\d+)\)\}", answer)
    if match:
        clue = match.group(1)
        number = int(match.group(2))
        return (clue, number)
    else:
        raise ValueError("The response format is incorrect")

def read_guess(answer):
    # Regular expression to match the format "{(one_word guess)}"
    match = re.search(r"\{\((\w+)\)\}", answer)
    if match:
        clue = match.group(1)
        return clue
    else:
        raise ValueError("The response format is incorrect")

In [ ]:
def play_game_GPT(lang, n, c, k, verbose = False):

  r = 1
  turn =  random.choice(["red", "blue"])
  board = generate_board(n, lang, c, k)
  points = {"red": 0, "blue": 0}

  if verbose: print(f"Initializing...")

  starting_prompt_red_master = START_master[lang] + team4prompt[(lang,"red")]
  starting_prompt_blue_master = START_master[lang] + team4prompt[(lang,"blue")]
  starting_prompt_red_guesser = START_guesser[lang] + team4prompt[(lang,"red")]
  starting_prompt_blue_guesser = START_guesser[lang] + team4prompt[(lang,"blue")]

  ### START CONVERSATION WITH RED MASTER WITH AN API TO GPT with starting_prompt_red_master
  ans = "..." ### INSERT ANSWER OF THE RED MASTER TO THE STARTING PROMPT
  if verbose:
    print(f"NARRATOR to MASTER(r): {starting_prompt_red_master}")
    print(f"MASTER(r): {ans}")

  ### START CONVERSATION WITH BLUE MASTER WITH AN API TO GPT with starting_prompt_blue_master
  ans = "..." ### INSERT ANSWER OF THE BLUE MASTER TO THE STARTING PROMPT
  if verbose:
    print(f"NARRATOR to MASTER(b): {starting_prompt_blue_master}")
    print(f"GUESSER(b): {ans}")

  ### START CONVERSATION WITH RED GUESSER WITH AN API TO GPT with starting_prompt_red_guesser
  ans = "..." ### INSERT ANSWER OF THE RED MASTER TO THE STARTING PROMPT
  if verbose:
    print(f"NARRATOR to GUESSER(r): {starting_prompt_red_guesser}")
    print(f"GUESSER(r): {ans}")

  ### START CONVERSATION WITH BLUE GUESSER WITH AN API TO GPT with starting_prompt_blue_guesser
  ans = "..." ### INSERT ANSWER OF THE BLUE GUESSER TO THE STARTING PROMPT
  if verbose:
    print(f"NARRATOR to GUESSER(b): {starting_prompt_blue_guesser}")
    print(f"GUESSER(b): {ans}")


  while True:
    if verbose: print(f" --- ROUND {r} ---")
    if verbose: print(f"NARRATOR: it is the turn of team {turn}.")

    # MASTER PHASE
    master_prompt = board4prompt(board, lang, master = True) +
    TURN_master[lang] + team4prompt[(lang, turn)] + FORMAT_master[lang]

    if verbose: print(f"NARRATOR to MASTER({turn}): {master_prompt}")
    ans = "..." ### INSERT ANSWER FROM THE API OF THE MASTER OF THE TURN WE ARE IN with master_prompt
    if verbose: print(f"MASTER({turn}): ans")

    try: clue, number = read_clue(ans)
    except:
      if verbose: print("!!! ERROR (Incorrect format of answer)")
      w = "blue" if turn == "red" else "red"
      return (w, "format", r)

    if clue in board:
      w = "blue" if turn == "red" else "red"
      if verbose: print(f"NARRATOR: MASTER({turn})'s clue was on the board, team {w} wins, game ends.")
      return (w, "said a word in the board as a hint", r)


    # GUESSER PHASE
    for iter in range(number):

      if iter == 1:
        guesser_prompt = TURN_guesser[lang] + board4prompt(board, lang) +
        MtoG_clue[lang] + clue + ". \n" + remider[lang] +
        +team4prompt[(lang, turn)] + FORMAT_guesser[lang]
      else:
        guesser_prompt = CORRECT_guesser[lang] + board4prompt(board, lang) +
        team4prompt[((lang,turn))] + MtoG_clue[lang] + clue + ". \n" + FORMAT_guesser[lang]

      if verbose: print(f"NARRATOR to GUESSER({turn}): {guesser_prompt}")
      ans = "..." ### INSERT ANSWER FROM THE API OF THE GUESSER OF THE TURN WE ARE IN with guesser_prompt
      if verbose: print(f"GUESSER({turn}): {ans}")
      try:
        guess = read_guess(ans).upper()
      except:
        if verbose: print("!!! ERROR (Incorrect format of answer)")
        w = "ble" if turn == "red" else "red"
        return (w, "format", r)

      #EVALUATE GUESS
      try:
        x = board[guess]
      except: # la guess non è nella board
        if verbose: print("!!! ERROR (Guess non in board)")
        w = "blue" if turn == "red" else "red"
        return (w, "said a word not in the board as a guess", r)

      if x == "killer":
        if verbose: print(f"NARRATOR: The killer word have been selected, the game ends.")
        w = "blue" if turn == "red" else "red"
        return (w, "killer", r)

      elif x == "blue":
        points["blue"] += 1
        if verbose: print(f'NARRATOR: A blue word have been selected (blue score = {points["blue"]}).')
        del board[guess]
        if points["blue"] == c:
          if verbose: print(f"NARRATOR: The blue team reached the goal, the game ends.")
          return ("blue", "win", r)

      elif x == "red":
        points["red"] += 1
        if verbose: print(f'NARRATOR: A red word have been selected (red score = {points["red"]}).')
        del board[guess]
        if points["red"] == c:
          if verbose: print(f"NARRATOR: The red team reached the goal, the game ends.")
          return ("red", "win", r)

      else:
        if verbose: print(f"NARRATOR: A neutral word have been selected.")

      if x[0] != turn: break

